In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.experiments import *
from LIMxCMBL.kernels import *
import matplotlib.pyplot as plt

In [2]:
nurest = cu.c/(21.106114054160 * u.cm)
Ofield = experiments['CHIME']['Omega_field']
dnu = (800 * u.MHz - 400 * u.MHz)/(256 * 4)

In [3]:
nuobs = np.linspace(617, 710,1000) * u.MHz
B = max(nuobs) - min(nuobs)
lobs = cu.c/nuobs
zobs = (nurest /nuobs - 1).si

In [4]:
Tsys = 55*u.K
Ssky = 31000 * u.deg**2

ttot = 1*u.yr
Nant = 256
npol = 2
Ncyl = 4
wcyl = 20*u.m
lcyl = 78*u.m

In [5]:
eta = 0.7
Ae = eta * lcyl/Nant * wcyl
Obeam = ((lobs**2/Ae)) * u.sr
du2 = (2*np.pi)**2/Obeam
du = np.sqrt(du2)

SFOV = (90*u.deg * (lobs/wcyl *(u.rad)))

In [6]:
bmin = 0.3048 * u.m
bmax = 102 * u.m
Db = (bmax - bmin)
umin = bmin / lobs * 1/u.rad
umax = bmax / lobs * 1/u.rad
Ndish = Nant * Ncyl

tp = ttot * SFOV / Ofield

In [7]:
n_u_z = Ndish * (Ndish - 1) / (2 * np.pi) / (umax**2 - umin**2)
N_dU = (n_u_z * du**2).si
print(N_dU[:10]) #independent of observing
N_dU_fid = N_dU[0]

N_u = ((umax-umin)/du).si

[2698.6195752 2698.6195752 2698.6195752 2698.6195752 2698.6195752
 2698.6195752 2698.6195752 2698.6195752 2698.6195752 2698.6195752]


In [8]:
tot_polar_bins = (np.pi*(umax**2-umin**2)/du**2).si 
#evaluate the sum in discrete case and pretends it holds in continous

In [9]:
((N_dU_fid  * tot_polar_bins)/(Ndish * (Ndish - 1)/2))
#total number of polar bins is consistent

<Quantity [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1.

In [10]:
PeT = (Tsys ** 2 
      / tp
      / dnu 
       / npol 
      / (N_dU_fid))

In [11]:
PeV = (PeT * Obeam**2 *
       (2 * cu.k_B * nuobs**2 / cu.c**2)**2 / u.sr**2).to(u.mJy**2)

In [12]:
PeI_dV = 1/(2*np.pi)**4 * np.pi * du**2 * (umax**2 - umin**2) * PeV

In [13]:
PeI = (PeI_dV 
       * dnu / nuobs 
       * nurest/nuobs 
       * Obeam/u.sr * Gamma_nu(nuobs, nurest)).to((u.kJy/u.sr)**2*u.Mpc**3)

In [14]:
PeI[499]

<Quantity 0.8319956 kJy2 Mpc3 / sr2>

In [15]:
u_f = u.Mpc**3 * (u.kJy/u.sr)**2

In [16]:
PeI_intuit = (
    (55 * u.K)**2 
    * 256
    /2
    /(4**2)
    *31000 * u.deg**2
    /(1*u.yr)
    *663*u.MHz
    *(1e4 * u.m**2)**2
    /(0.7)**3
    /(78*u.m)**3
    /(20*u.m)**2
    *32*u.Gpc**3 
    #after this its prefactors
    *0.63661977 #basically 1/90deg in rads
    *cu.c
    *nurest
    *(2*cu.k_B/cu.c**2/u.sr)**2
    /(2*np.pi)**2
    /u.sr
    /2 # fixed factor of two
).to(u_f)
print(PeI_intuit)
(PeI_intuit/PeI[499]).si

0.7647520931345883 kJy2 Mpc3 / sr2


<Quantity 0.91917805>

In [17]:
PeI_intuit.to(u_f)

<Quantity 0.76475209 kJy2 Mpc3 / sr2>

In [18]:
PeI_intuit2 = (
    (Tsys)**2 
    * Nant
    /npol
    /(Ncyl**2)
    *Ofield
    /ttot
    *nuobs
    *(bmax**2 - bmin**2)**2
    /(eta)**3
    /(lcyl)**3
    /(wcyl)**2
    *Gamma_nu(nuobs,nurest)
    #after this its prefactors
    *0.63661977 #mathematica, basically 1/90deg
    *cu.c
    *nurest
    *(2*cu.k_B/cu.c**2/u.sr)**2
    /(2*np.pi)**2
    /u.sr
    /2 #fixed factor of 2
).si#to((u.kJy/u.sr)**2 * u.Mpc**3)
((PeI_intuit2-PeI)/PeI).si

<Quantity [-0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -0.00097657, -0.00097657, -0.00097657, -0.00097657, -0.00097657,
           -

In [19]:
PeI_intuit2[499].to(u_f)

<Quantity 0.8311831 kJy2 Mpc3 / sr2>

In [27]:
PeI_intuit3 = (
    (Tsys/(55 * u.K))**2 
    * 256/Nant
    /(npol/2)
    /(Ncyl/4)**2
    *Ofield/(31000 * u.deg**2)
    /(ttot/(1*u.yr))
    *(nuobs/(663*u.MHz))
    *((bmax**2-bmin**2)/(1e4 * u.m**2))**2
    /(eta/0.7)**3
    /(lcyl/(78*u.m))**3
    /(wcyl/(20*u.m))**2
    *(Gamma_nu(nuobs,nurest)/(32*u.Gpc**3)) 
)*1.53 * u_f/2 #fix factor of 2

((PeI_intuit3-PeI)/PeI).si

<Quantity [-0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -0.00065272, -0.00065272, -0.00065272, -0.00065272, -0.00065272,
           -

In [28]:
PeI_intuit3[499].to(u_f)

<Quantity 0.83145254 kJy2 Mpc3 / sr2>

In [29]:
PeI[499]

<Quantity 0.8319956 kJy2 Mpc3 / sr2>

In [30]:
PeI_intuit3[499].to(u.Mpc**3 * (u.kJy/u.sr)**2)

<Quantity 0.83145254 kJy2 Mpc3 / sr2>

In [31]:
a,b = CHIME_Pei()

In [32]:
b/PeI

<Quantity [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1.

In [33]:
b[499]

<Quantity 0.8319956 kJy2 Mpc3 / sr2>

In [34]:
1.53 * u_f/2 #fix factor of 2

<Quantity 0.765 kJy2 Mpc3 / sr2>